Let's try to load the query logs

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from os import path

In [2]:
# steps to create a unified logs file.
def create_unified_log():
    if (not path.exists('data/logs.csv')):
        logs1 = pd.read_csv('data/logs/Clean-Data-01.txt', sep='\t')
        logs2 = pd.read_csv('data/logs/Clean-Data-02.txt', sep='\t')
        logs3 = pd.read_csv('data/logs/Clean-Data-03.txt', sep='\t')
        logs4 = pd.read_csv('data/logs/Clean-Data-04.txt', sep='\t')
        logs5 = pd.read_csv('data/logs/Clean-Data-05.txt', sep='\t')
        frames = [logs1, logs2, logs3, logs4, logs5]
        logs = pd.concat(frames) 
        # AnonID is not a unique column, so it would not be a good idea to make it an index
        #logs['AnonID'].nunique() #238,544
        #len(logs['AnonID']) # 394,2354

        # let's save it to a csv file
        logs.to_csv('data/logs.csv', index=False)
create_unified_log()

In [3]:
logs = pd.read_csv('data/logs.csv')
logs['Query'] = logs['Query'].str.strip()
logs = logs.rename(columns={'AnonID': 'UserId'})
logs.head()
# 3,942,354 queries

,UserId,Query,QueryTime
0,142,merit release appearance,2006-04-22 23:51:18
1,217,lottery,2006-03-01 11:58:51
2,217,lottery,2006-03-27 14:10:38
3,217,vietnam,2006-05-22 17:43:42
4,217,vietnam,2006-05-22 18:03:24


In [4]:
# set the maximum frequency of occurrence of any query in QL
#logs.groupby('Query').count()[['Id']].sort_values('Id', ascending=False)
# the query google has the max frecuency, which is 83677
#logs.groupby('Query').count()[['Id']].sort_values('Id', ascending=False) #.iloc(0)[0][0]
#max_frequency = logs.groupby('Query').count()[['Id']].sort_values('Id', ascending=False).iloc(0)[0][0]
max_frequency = 83677 # hard code it to improve performance

In [14]:
# set query
query = 'google'

In [72]:
# gets a subset from the query
#query_filters = logs[(logs['Query'].str.startswith(query, na=False)) & (logs['Query'].str.len() > len(query) - 1)]
query_base_candidates = logs[logs['Query'] == query]

#query_no_duplicates = query_filters.drop_duplicates(subset='Query').sort_values('Query')
##len(query_no_duplicates['Query'].unique())
#number_sessions_for_query = len(query_no_duplicates)
#query_no_duplicates
query_base_candidates

,UserId,Query,QueryTime
38,1410,google,2006-05-01 21:40:54
39,2005,google,2006-03-24 21:25:10
49,2178,google,2006-03-27 20:58:44
55,2178,google,2006-04-11 11:06:20
68,2178,google,2006-05-16 10:54:39
87,2421,google,2006-05-04 15:39:25
90,2421,google,2006-05-04 21:14:33
91,2421,google,2006-05-05 16:16:01
146,2722,google,2006-04-12 15:18:12
195,2722,google,2006-05-02 09:09:19


In [73]:
# Get query candidates for suggestions, the ones which in the same session changed from query text to query text + something else
#i = 0
#candidates_counter = {}
arr_candidate_queries = []
count = 0
# loop through query_base_candidates to get the real candidates 
for index, row in query_base_candidates.iterrows():
    user_id = row[0]
    query = row[1]
    current_index = index + 1
    current_user_id = logs.loc[current_index, 'UserId']
    has_next = False
    is_invalid = False
    while (user_id == current_user_id):
        # check for current query text 
        current_query = logs.loc[current_index, 'Query']
        if (not pd.isna(current_query) and not current_query.startswith(query)):
            is_invalid = True
            break
        if (not pd.isna(current_query) and current_query.startswith(query) and len(current_query) > len(query)):
            has_next = True
        # move to next row    
        current_index += 1
        current_user_id = logs.loc[current_index, 'UserId']
    
    if (has_next and is_invalid == False):
        # process current_index - 1
        current_index -= 1
        current_query = logs.loc[current_index, 'Query']
        if (not pd.isna(current_query) and current_query.startswith(query) and len(current_query) > len(query)):
            count += 1
            arr_candidate_queries.append(logs.loc[current_index])
            #if (current_query in candidates_counter):
            #    candidates_counter[current_query] += 1
            #else:
            #    candidates_counter[current_query] = 1          
#    print(row[0])
#    print(logs.loc[index, 'UserId'])
#    i += 1
#    if (i == 10): break
#        if (isinstance(current_query, float)):
#            print(logs.loc[current_index])
#print(count)
#print(candidate_queries)
candidate_queries = pd.DataFrame(arr_candidate_queries)
candidate_queries

,UserId,Query,QueryTime
21635,222187,google search,2006-04-02 10:42:58
30747,324779,google information about people,2006-05-09 20:48:31
49868,533611,google zip code,2006-05-30 21:55:10
146535,1654628,google search,2006-05-18 01:43:35
328367,3678892,google maps,2006-03-27 20:27:41
334802,3807683,google images,2006-05-19 15:12:39
374973,4539882,google better business bureau,2006-05-16 22:42:00
397458,4977135,google white pages,2006-03-02 15:56:16
406599,5134637,google toolbar,2006-04-23 19:14:47
452099,6094698,google search engine,2006-05-23 18:48:02


In [74]:
# set the frequency - 𝐹𝑟𝑒𝑞(𝐶𝑄)
query_counts = logs[logs['Query'].isin(candidate_queries['Query'])].groupby('Query')['Query'].count()
query_results = candidate_queries.join(query_counts, on='Query', lsuffix='_text')
query_results = query_results.rename(columns={'Query': 'Count', 'Query_text': 'Query'})
query_results['Freq'] = query_results['Count'] / max_frequency
query_results

,UserId,Query,QueryTime,Count,Freq
21635,222187,google search,2006-04-02 10:42:58,939,0.011222
30747,324779,google information about people,2006-05-09 20:48:31,1,0.000012
49868,533611,google zip code,2006-05-30 21:55:10,1,0.000012
146535,1654628,google search,2006-05-18 01:43:35,939,0.011222
328367,3678892,google maps,2006-03-27 20:27:41,634,0.007577
334802,3807683,google images,2006-05-19 15:12:39,310,0.003705
374973,4539882,google better business bureau,2006-05-16 22:42:00,2,0.000024
397458,4977135,google white pages,2006-03-02 15:56:16,18,0.000215
406599,5134637,google toolbar,2006-04-23 19:14:47,18,0.000215
452099,6094698,google search engine,2006-05-23 18:48:02,43,0.000514


In [75]:
# set the mod - 𝑀𝑜𝑑(𝐶𝑄,𝑞′)
sessions_count = candidate_queries.groupby('Query')['Query'].count().astype(object)

#query_counts_sessions = query_filters.groupby('Query')['Id'].nunique()

#query_results = query_results.join(query_counts_sessions, on='Query', lsuffix='_text').rename(columns={'Id': 'SessionCount', 'Id_text': 'Id'})
query_results = query_results.join(sessions_count, on='Query', lsuffix='_text').rename(columns={'Query': 'CountInSession', 'Query_text': 'Query'})
# TODO: check if len(candidate_queries) is the correct number to use based on "the total number of sessions in QL in which q’ appears. " 
query_results['Mod'] = query_results['CountInSession'] / len(candidate_queries) 
query_results

,UserId,Query,QueryTime,Count,Freq,CountInSession,Mod
21635,222187,google search,2006-04-02 10:42:58,939,0.011222,16,0.148148
30747,324779,google information about people,2006-05-09 20:48:31,1,0.000012,1,0.00925926
49868,533611,google zip code,2006-05-30 21:55:10,1,0.000012,1,0.00925926
146535,1654628,google search,2006-05-18 01:43:35,939,0.011222,16,0.148148
328367,3678892,google maps,2006-03-27 20:27:41,634,0.007577,5,0.0462963
334802,3807683,google images,2006-05-19 15:12:39,310,0.003705,2,0.0185185
374973,4539882,google better business bureau,2006-05-16 22:42:00,2,0.000024,1,0.00925926
397458,4977135,google white pages,2006-03-02 15:56:16,18,0.000215,1,0.00925926
406599,5134637,google toolbar,2006-04-23 19:14:47,18,0.000215,2,0.0185185
452099,6094698,google search engine,2006-05-23 18:48:02,43,0.000514,5,0.0462963


In [24]:
# set the time - 𝑇𝑖𝑚𝑒(𝐶𝑄, 𝑞′)
#convert to datetime
query_results['QueryDateTime'] = pd.to_datetime(query_results['QueryTime'], format='%Y-%m-%d %H:%M:%S')
all_queries_max_date = query_results.groupby('Query')['QueryDateTime'].max()
query_results = query_results.join(all_queries_max_date, on='Query', lsuffix='_text').rename(columns={'QueryDateTime': 'MaxDateBySession'})
query_results = query_results.drop(columns=['QueryDateTime_text'])
query_results['Time'] = (pd.to_datetime('today') - query_results['MaxDateBySession']) / np.timedelta64(1, 'h')


,UserId,Query,QueryTime
1843957,3324224,google addresses,2006-05-19 12:38:35
2845762,7402440,google addresses,2006-05-20 13:20:54
2845764,7402440,google addresses,2006-05-20 13:37:03
2845767,7402440,google addresses,2006-05-20 13:42:46
2993602,12861248,google addresses,2006-05-28 12:26:49


In [49]:
logs[logs['Query'].isin(candidate_queries['Query'])].groupby('Query')['Query'].count()

Query
google acting                                1
google addresses                             5
google airlines                              1
google better business bureau                2
google blue book value                       1
google car insurance                         2
google car parts                             1
google confederacy of dunces review          2
google crafts                                1
google earth                              1317
google failure                               4
google find this person                      2
google free nursing information              1
google free xxx                              2
google fundraisers                           1
google games                                58
google goose down comforters                 1
google graph satellite images                1
google home page                             9
google honda                                 1
google image                                99
google 

In [43]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3942354 entries, 0 to 3942353
Data columns (total 3 columns):
UserId       int64
Query        object
QueryTime    object
dtypes: int64(1), object(2)
memory usage: 90.2+ MB


In [50]:
#logs['Query'] = logs['Query'].astype(object)
#candidate_queries['Query'] = candidate_queries['Query'].astype(object)

#logs.join(candidate_queries, on='Query', lsuffix='_text').groupby('Query')['Query'].count()
logs[logs['Query'].isin(candidate_queries['Query'])].groupby('Query')['Query'].count()

Query
google acting                                1
google addresses                             5
google airlines                              1
google better business bureau                2
google blue book value                       1
google car insurance                         2
google car parts                             1
google confederacy of dunces review          2
google crafts                                1
google earth                              1317
google failure                               4
google find this person                      2
google free nursing information              1
google free xxx                              2
google fundraisers                           1
google games                                58
google goose down comforters                 1
google graph satellite images                1
google home page                             9
google honda                                 1
google image                                99
google 

In [22]:
query_counts = candidate_queries.groupby('Query')['Query'].count()
query_counts


Query
google acting                              1
google addresses                           3
google airlines                            2
google better business bureau              1
google blue book value                     1
google car insurance                       2
google car parts                           1
google confederacy of dunces review        1
google crafts                              1
google earth                              13
google failure                             1
google find this person                    1
google free nursing information            1
google free xxx                            1
google fundraisers                         4
google games                               1
google goose down comforters               1
google graph satellite images              1
google home page                           2
google honda                               1
google image                               1
google images                              2
goog

In [ ]:





# set the min values
min_freq = query_results['Freq'].min()
min_mod = query_results['Mod'].min()
min_time = query_results['Time'].min()

query_results['Score'] = (query_results['Freq'] + query_results['Mod'] + query_results['Time']) / 1 - (min_freq + min_mod + min_time)

query_results.sort_values('Score', ascending=False)